In [ ]:
import csv
import base64

import requests
import urllib.request

# 1. Authentification

Setup API entry point at https://developer.spotify.com/dashboard/.

Get *CLIENT_ID* and *CLIENT_SECRET* (enter below) and add `https://127.0.0.1/` as callbacks URI.

In [ ]:
CLIENT_ID = ""
CLIENT_SECRET = ""

In [ ]:
URL = "https://api.spotify.com/v1/"
AUTH_URL = 'https://accounts.spotify.com/authorize'
TOKEN_URL = 'https://accounts.spotify.com/api/token'

In [ ]:
auth_code = requests.get(AUTH_URL, {
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': 'https://127.0.0.1/',
    'scope': 'user-library-read user-follow-read playlist-read-collaborative playlist-read-private'
})

print(auth_code.url)

Approve the application with the link above and get the auth code in the callback URL :
> https://127.0.0.1/?code=XXXX

Paste the code in the cell below.

In [ ]:
CODE = ""

In [ ]:
auth_header = base64.urlsafe_b64encode((CLIENT_ID + ':' + CLIENT_SECRET).encode())

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': 'Basic %s' % auth_header.decode('ascii')
}

payload = {
    'grant_type': 'authorization_code',
    'code': CODE, 
    'redirect_uri': 'https://127.0.0.1/',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
}

# Make a request to the /token endpoint to get an access token
access_token_request = requests.post(url=TOKEN_URL, data=payload, headers=headers)

# convert the response to JSON
access_token = access_token_request.json()['access_token']


headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}


# 2. Fetch library data

In [ ]:
def fetch_all(endpoint, headers):
    items = []
    r = requests.get(endpoint, headers=headers).json()
    items += (r["items"])
    while r["next"]:
        r = requests.get(r["next"], headers=headers).json()
        items += (r["items"])
    return items

def fetch_all_artists(endpoint, headers):
    items = []
    r = requests.get(endpoint, headers=headers).json()["artists"]
    items += (r["items"])
    while r["next"]:
        r = requests.get(r["next"], headers=headers).json()["artists"]
        items += (r["items"])
    return items

In [ ]:
playlists = fetch_all(URL + "me/playlists", headers)
tracks = fetch_all(URL + "me/tracks", headers)
shows = fetch_all(URL + "me/shows", headers)
albums = fetch_all(URL + "me/albums", headers)
artists = fetch_all_artists(URL + "me/following?type=artist", headers)

# 3. Export to CSV

In [ ]:
PATH = "./SAVE/"

In [ ]:
with open(PATH + "artists.csv", 'w') as f:
    c = csv.writer(f)
    c.writerow(["uri", "name"])
    for i in artists:
        c.writerow([i["uri"], i["name"]])

In [ ]:
with open(PATH + "albums.csv", 'w') as f:
    c = csv.writer(f)
    c.writerow(["uri", "name", "artists", "added_at", "total_tracks", "release_date"])
    for i in albums:
        artist = ""
        for j in i["album"]["artists"]:
            artist += ";" if len(artist) else ""
            artist += j["name"]

        c.writerow([i["album"]["uri"], i["album"]["name"], artist, i["added_at"], i["album"]["total_tracks"], i["album"]["release_date"]])
        

In [ ]:
with open(PATH + "shows.csv", 'w') as f:
    c = csv.writer(f)
    c.writerow(["uri", "name", "publisher", "added_at", "total_episodes"])
    for i in shows:
        c.writerow([i["show"]["uri"], i["show"]["name"], i["show"]["publisher"], i["added_at"], i["show"]["total_episodes"]])

In [ ]:
with open(PATH + "playlists.csv", 'w') as f:
    c = csv.writer(f)
    c.writerow(["uri", "name", "owner_uri", "owner_name", "total_tracks", "public", "description"])
    for i in playlists:
        c.writerow([i["uri"], i["name"], i["owner"]["uri"], i["owner"]["display_name"], i["tracks"]["total"], i["public"], i["description"]])
        urllib.request.urlretrieve(i["images"][0]["url"], PATH + "IMG/" + i["uri"] + ".jpg")

In [ ]:
cache_user_id = {}

for p in playlists:
    with open(PATH + "PLAYLISTS/" + p["id"] + ".csv", 'w') as f:
        c = csv.writer(f)
        tracklist = fetch_all(URL + "playlists/" + p["id"] + "/tracks", headers)
        for t in tracklist:
            artist = ""
            for j in t["track"]["artists"]:
                artist += ";" if len(artist) else ""
                artist += j["name"]
            if t["added_by"]["id"] not in cache_user_id:
                r = requests.get(URL + "users/" + t["added_by"]["id"], headers=headers)
                cache_user_id[t["added_by"]["id"]] = "Undefined" if r.status_code > 400 else r.json()["display_name"] 
                
            c.writerow([t["track"]["uri"], t["track"]["name"], artist, t["added_at"], t["added_by"]["uri"], cache_user_id[t["added_by"]["id"]]])